# Gene regulatory network construction model

In [2]:
# =============================================================================
# 0.  Imports and paths
# =============================================================================
import os, sys, time, json
import numpy  as np
import pandas as pd
import scanpy as sc

import matplotlib.pyplot as plt
import matplotlib as mpl
import os, gc, warnings

import scipy.io as sio
from scipy import sparse
import anndata as ad

import celloracle            as co
from   celloracle import motif_analysis as ma

co.__version__

which: no R in (/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin)


'0.20.0'

In [21]:
# — user parameters — 
REF_GENOME         = "hg38"                           # or "mm10", "hg19"
PEAK2GENE_CSV      = "peak2gene_archr_allGenes.csv"            # ArchR peak→gene links
MOTIF_POSITIONS_CSV= "archr_motif_positions.csv"      # ArchR peak↔motif table
BASE_GRN_PARQUET   = "base_GRN_dataframe.parquet"     # final output
CLUSTER_COL        = "seurat_clusters"                       # adjust to your adata.obs column
EMBEDDING_KEY      = "X_umap"                         # adjust to your adata.obsm key

# ---------------------------------------------------------------------------
# 1.  Load peak→gene links (ArchR)
# ---------------------------------------------------------------------------
peak2gene = pd.read_csv(PEAK2GENE_CSV)
print(f"peak2gene rows: {peak2gene.shape[0]}")



peak2gene rows: 80974


In [15]:
# ---------------------------------------------------------------------------
# 2.  Make TFinfo
# ---------------------------------------------------------------------------
tfi = ma.TFinfo(
    peak_data_frame = peak2gene,
    ref_genome      = REF_GENOME
)

# ╔═════════════════════════════════════════════════════════════════════════╗
# ║  OPTION A – run CellOracle’s own motif scan (simplest, but slow)        ║
# ╚═════════════════════════════════════════════════════════════════════════╝
RUN_INTERNAL_SCAN = True        # set to False if you prefer option B

if RUN_INTERNAL_SCAN:
    print("⇢ running CellOracle motif scan (can take hours)…")
    tfi.scan(fpr=0.02, motifs=None, verbose=True)   # default gimme.vertebrate v5
else:
    # ╔═════════════════════════════════════════════════════════════════════╗
    # ║  OPTION B – insert ArchR motif positions                            ║
    # ╚═════════════════════════════════════════════════════════════════════╝
    # MOTIF_POSITIONS_CSV must have: peak_id, motif_id, score
    archr = pd.read_csv(MOTIF_POSITIONS_CSV)
    # Build a minimal “scanned_df” that TFinfo expects.
    #  Required columns: seqname, motif_id, factors_direct, factors_indirect,
    #                    score, pos, strand
    archr_scanned = pd.DataFrame({
        "seqname"          : archr.peak_id,
        "motif_id"         : archr.motif_id,
        # leave factors_direct / indirect empty – CellOracle doesn’t need them
        "factors_direct"   : "",
        "factors_indirect" : "",
        "score"            : archr.score,
        "pos"              : -1,     # unknown motif position inside peak
        "strand"           : 0       # placeholder
    })
    tfi.scanned_df = archr_scanned
    print(f"Injected {len(archr_scanned):,} ArchR motif hits into TFinfo")

# ---------------------------------------------------------------------------
# 3.  Filter motif hits & produce base-GRN
# ---------------------------------------------------------------------------
tfi.filter_motifs_by_score(threshold=10)
tfi.make_TFinfo_dataframe_and_dictionary(verbose=True)

base_grn = tfi.to_dataframe()
base_grn.to_parquet(BASE_GRN_PARQUET)
print("✔ base_GRN_dataframe.parquet written — shape:", base_grn.shape)


⇢ running CellOracle motif scan (can take hours)…
No motif data entered. Loading default motifs for your species ...
 Default motif for vertebrate: gimme.vertebrate.v5.0. 
 For more information, please see https://gimmemotifs.readthedocs.io/en/master/overview.html 

Initiating scanner... 



2025-06-13 18:27:51,087 - DEBUG - using background: genome hg38 with size 200


Calculating FPR-based threshold. This step may take substantial time when you load a new ref-genome. It will be done quicker on the second time. 



2025-06-13 18:28:47,839 - DEBUG - determining FPR-based threshold


Motif scan started .. It may take long time.



...

  0%|          | 0/22214 [00:00<?, ?it/s]

2. Converting results into dictionaries.


  0%|          | 0/15477 [00:00<?, ?it/s]

  0%|          | 0/1095 [00:00<?, ?it/s]

✔ base_GRN_dataframe.parquet written — shape: (80974, 1098)


In [3]:
# 1) Load your base GRN
base_grn = pd.read_parquet("base_GRN_dataframe.parquet")
print("base GRN shape:", base_grn.shape)
# e.g. (80974, 1098)

base GRN shape: (80974, 1098)
